<a href="https://colab.research.google.com/github/NicolaGabriele/MastodonContentCompliance/blob/main/topic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 66.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.6 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8


In [2]:
import os
from bertopic import BERTopic
import json
RESULTS_HOME = '/kaggle/input/instance-json/results/results'
topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

2024-07-04 08:29:27.250092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 08:29:27.250203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 08:29:27.382350: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


topics.json:   0%|          | 0.00/10.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

topic_embeddings.safetensors:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

ctfidf_config.json:   0%|          | 0.00/4.66M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
'''topic, prob = topic_model.transform("This is an incredible movie!")
topic_model.topic_labels_[topic[0]]'''
from tqdm import tqdm

In [5]:
instances_file_path = '/kaggle/input/instance-json/instances.jsonl'

instances = []

with open(instances_file_path, 'r') as file:
    for line in file:
        instance_data = json.loads(line)
        instances.append(instance_data)

filtered_instances = [inst for inst in instances if inst["instance"] != "mastodon.social"]

valid_instances = []

for instance in filtered_instances:
    instance_file_path = os.path.join(RESULTS_HOME, f"{instance['instance']}.json")
    if os.path.isfile(instance_file_path):
        valid_instances.append(f"{instance['instance']}.json")
    if len(valid_instances) == 15:
        break

print(valid_instances)


['good.news.json', 'pawoo.net.json', 'test.pawoo.org.json', 'baraag.net.json', 'mstdn.social.json', 'pravda.me.json', 'mastodon.online.json', 'mastodon.world.json', 'mas.to.json', 'kids.0px.io.json', 'techhub.social.json', 'universeodon.com.json', 'mastodonapp.uk.json', 'masto.ai.json', 'mastodon.uno.json']


In [6]:
def process_batch(posts_batch):
    t, _ = topic_model.transform(posts_batch)
    return [topic_model.topic_labels_[i] for i in t]

In [ ]:

for instance_name in tqdm(valid_instances, desc="Processing Instance"):
    aligned_count = 0
    total_count = 0
    instance = json.load(open(os.path.join(RESULTS_HOME, instance_name), 'r'))
    posts = [post['text'] for post in tqdm(instance['records'], desc='Pull post' ) if post['language'] == 'en']
    aux = [post['tags'] for post in instance['records'] if post['language'] == 'en']
    tags = []
    for tagl in tqdm(aux, desc='Processing tags'):
        for tag in tagl:
            tags.append(tag['name'])
    
    #print(f"{instance_name}",tags)
    
    batch_size = 10000
    batches = [posts[i:i + batch_size] for i in range(0, len(posts), batch_size)]

    topic_path = os.path.join('/kaggle/working', f'fullgas_topics_{instance_name}.txt')
    
    all_topics = []
    
    if os.path.exists(topic_path):
        # Se il file esiste, carica i topic dal file
        with open(topic_path, 'r') as f:
            all_topics = [line.strip() for line in f.readlines()]
        print(f'Topics loaded from {topic_path}')
    else:
        # Se il file non esiste, genera i topic e salvali nel file
        for batch in tqdm(batches, desc='Processing batches'):
            topics = process_batch(batch)
            all_topics.extend(topics)

            with open(topic_path, 'a') as f:
                for topic in topics:
                    f.write(f'{topic}\n')

    if len(tags) > 0:
        total_count += len(tags)
        for tag in tqdm(tags, desc='Checking tag alignment'):
            for topic in all_topics:
                if tag in topic:
                    aligned_count += 1
                    break
    else:
        total_count = 1
    output_path = os.path.join('/kaggle/working', f'count_results{instance_name}.txt')
    with open(output_path, 'w') as file:
        file.write(f'Aligned Count: {aligned_count}\n')
        file.write(f'Total Count: {total_count}\n')
        file.write(f'Alignment / Total : {aligned_count / total_count:.4f}\n')
                
    print(aligned_count)
    print(total_count)
    print(aligned_count/total_count)

Processing Instance:   0%|          | 0/15 [00:00<?, ?it/s]
Pull post: 0it [00:00, ?it/s]

Processing tags: 0it [00:00, ?it/s]

Processing batches: 0it [00:00, ?it/s]


0
1
0.0



Pull post: 100%|██████████| 50000/50000 [00:00<00:00, 1171547.48it/s]

Processing tags: 100%|██████████| 19270/19270 [00:00<00:00, 302695.50it/s]


Topics loaded from /kaggle/working/fullgas_topics_pawoo.net.json.txt



Processing Instance:  13%|█▎        | 2/15 [09:57<1:04:42, 298.68s/it]

...

2024-07-04 10:08:02,784 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.

Processing batches:  60%|██████    | 3/5 [00:16<00:10,  5.42s/it]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2024-07-04 10:08:08,425 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.

Processing batches:  80%|████████  | 4/5 [00:21<00:05,  5.51s/it]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

2024-07-04 10:08:11,437 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.

Processing batches: 100%|██████████| 5/5 [00:24<00:00,  4.98s/it]

Checking tag alignment:  97%|█████████▋| 91389/94682 [08:12<00:16, 195.31it/s]


In [15]:
instance = json.load(open(os.path.join(RESULTS_HOME, 'mastodon.social.json'), 'r'))
print(len(tags))
print(len(tags2))

400000
414704


In [6]:
from tqdm import tqdm
c = 0
for instance_name in tqdm(os.listdir(RESULTS_HOME)):
    instance = json.load(open(os.path.join(RESULTS_HOME, instance_name), 'r'))
    posts = [post['text'] for post in instance['records'] if post['language'] == 'en']
    aux = [post['tags'] for post in instance['records'] if post['language'] == 'en']
    tags = []
    for tagl in aux:
        for tag in tagl:
            tags.append(tag['name'])
    if len(tags)>0:
        c+= len(posts)
    
print(c)
#Numero di post totali da analizzare 18387780

100%|██████████| 788/788 [12:02<00:00,  1.09it/s]

18387780


In [15]:
#!rm -r /kaggle/working
#!zip -r scores.zip /kaggle/working

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/count_resultsgood.news.json.txt (deflated 22%)
  adding: kaggle/working/fullgas_topics_mastodon.online.json.txt (deflated 99%)
  adding: kaggle/working/fullgas_topics_pravda.me.json.txt (deflated 99%)
  adding: kaggle/working/count_resultsmasto.ai.json.txt (deflated 16%)
  adding: kaggle/working/count_resultsuniverseodon.com.json.txt (deflated 17%)
  adding: kaggle/working/count_resultspawoo.net.json.txt (deflated 17%)
  adding: kaggle/working/count_resultsmastodonapp.uk.json.txt (deflated 18%)
  adding: kaggle/working/count_resultsmas.to.json.txt (deflated 18%)
  adding: kaggle/working/count_resultsmastodon.uno.json.txt (deflated 17%)
  adding: kaggle/working/count_resultsbaraag.net.json.txt (deflated 21%)
  adding: kaggle/working/count_resultskids.0px.io.json.txt (deflated 22%)
  adding: kaggle/working/fullgas_topics_mas.to.json.txt (deflated 99%)
  adding: kaggle/working/count_resultsmstdn.social.json.txt (deflated 17%)
